In [32]:
###############################################################
# 6-vertex connected graphs → images + automorphisms + A-matrix
# Everything in ONE PDF (patterns first, matrix next)
###############################################################

from sage.graphs.graph_generators import graphs
from itertools import permutations
from PIL import Image
import tempfile, os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from sage.graphs.graph import Graph

###############################################################
# Collect graphs (connected) 
###############################################################

G6 = [G for G in graphs(6) if G.is_connected()]
m = len(G6)

labels = [f"G{i+1:03d}" for i in range(m)]
aut = [int(G.automorphism_group().cardinality()) for G in G6]
# ------------------------------------------------------------
# Sort graphs by number of automorphisms (increasing)
# ------------------------------------------------------------
order = sorted(range(m), key=lambda i: aut[i])

G6     = [G6[i] for i in order]
labels = [labels[i] for i in order]
aut    = [aut[i] for i in order]

print("Total connected graphs on 6 vertices:", m)

###############################################################
# Creating PNG for each graph
###############################################################

img_files = []
for i, G in enumerate(G6, start=1):
    tmp = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    tname = tmp.name
    tmp.close()

    G.plot(figsize=3).save(tname)

    img = Image.open(tname).convert("RGBA")
    img.thumbnail((250, 250), Image.LANCZOS)

    out = f"p6_{i:03d}.png"
    img.save(out)
    img_files.append(out)

    os.remove(tname)

###############################################################
# Induced copy count (injective homomorphisms)
###############################################################
def embedding_count(P, H):
    k = P.order()
    P_vertices = list(P.vertices())
    H_vertices = list(H.vertices())

    P_edges = {(min(u,v), max(u,v)) for u,v,_ in P.edges()}

    count = 0

    # Choose k distinct vertices of H, then try all injections
    for S in permutations(H_vertices, k):
        mapping = dict(zip(P_vertices, S))

        ok = True
        for u, v in P_edges:
            if not H.has_edge(mapping[u], mapping[v]):
                ok = False
                break

        if ok:
            count += 1

    return count


# Build A matrix: A[i][j] = number of (induced) copies of G_i in G_j
A = [[0]*m for _ in range(m)]
for i in range(m):
    temp = aut[i]
    for j in range(m):
        A[i][j] = embedding_count(G6[i], G6[j]) // temp


###############################################################
# Build one PDF combining gallery + matrix
###############################################################

pdf = "p6_full_corrected.pdf"
c = canvas.Canvas(pdf, pagesize=letter)
pw, ph = letter

margin = 0.5*inch
gap = 0.3*inch
cell_w = (pw - 2*margin - gap)/2
cell_h = (ph - 2*margin - gap)/2
header_h = 0.35*inch

# ---------------- Graph gallery (2×2 per page) ----------------
for idx in range(m):
    pos = idx % 4
    row = pos // 2
    col = pos % 2

    if pos == 0 and idx != 0:
        c.showPage()

    tx = margin + col*(cell_w + gap)
    top = ph - margin - row*(cell_h + gap)
    ty = top - cell_h

    c.setFont("Helvetica-Bold", 9)
    c.drawString(tx+5, top-12, f"{labels[idx]}   Aut: {aut[idx]}")

    box_w = cell_w - 0.2*inch
    box_h = cell_h - header_h - 0.2*inch

    img = Image.open(img_files[idx])
    w, h = img.size
    scale = min(box_w/w, box_h/h)
    dw, dh = w*scale, h*scale

    dx = tx + (cell_w - dw)/2
    dy = ty + (cell_h - header_h - dh)/2

    c.drawImage(img_files[idx], dx, dy, width=dw, height=dh)

c.showPage()

# ---------------- Matrix pages ----------------
c.setFont("Courier", 6)

x0 = 30
y0 = ph - 40
c.drawString(x0, y0, "A[i][j] = number of induced copies of Gi in Gj")
y = y0 - 20

row_h = 10
header = "     " + " ".join(f"{lab:>5}" for lab in labels)
c.drawString(x0, y, header)
y -= row_h

for i in range(m):
    row = f"{labels[i]:<5}" + " ".join(f"{A[i][j]:5d}" for j in range(m))
    c.drawString(x0, y, row)
    y -= row_h

    if y < 40:
        c.showPage()
        y = ph - 60
        c.setFont("Courier", 6)

c.showPage()
c.save()

print("DONE. Output PDF:", pdf)


Total connected graphs on 6 vertices: 112
DONE. Output PDF: p6_full_corrected.pdf


In [7]:
from sage.graphs.graph_generators import graphs

# Connected graphs on 6 vertices, ordered by automorphisms
G6 = [G for G in graphs(6) if G.is_connected()]
aut = [int(G.automorphism_group().cardinality()) for G in G6]
order = sorted(range(len(G6)), key=lambda i: aut[i])
G6 = [G6[i] for i in order]
print(len(G6))
# Fixed edge order
edge_order = [
    (0,1), (0,2), (0,3), (0,4),(0,5),
    (1,2), (1,3), (1,4),(1,5),
    (2,3), (2,4),(2,5),
    (3,4),(3,5),
    (4,5)
]

def edge_bitstring(G):
    return ''.join('1' if G.has_edge(u,v) else '0' for u,v in edge_order)

# Print ONLY matrix entries
for i, G in enumerate(G6, start=1):
    print(f"G{i} & {edge_bitstring(G)} &  &  &  &  \\\\")

112
G1 & 001110011001010 &  &  &  &  \\
G2 & 001110011001001 &  &  &  &  \\
G3 & 001010011010001 &  &  &  &  \\
G4 & 001110011010011 &  &  &  &  \\
G5 & 001100101010011 &  &  &  &  \\
G6 & 001110101011010 &  &  &  &  \\
G7 & 001110101010011 &  &  &  &  \\
G8 & 001110101011110 &  &  &  &  \\
G9 & 000110010001010 &  &  &  &  \\
G10 & 000110010001011 &  &  &  &  \\
G11 & 001010011001000 &  &  &  &  \\
G12 & 001010011001010 &  &  &  &  \\
G13 & 001110011001000 &  &  &  &  \\
G14 & 001100011001010 &  &  &  &  \\
G15 & 001110011001011 &  &  &  &  \\
G16 & 001010011010000 &  &  &  &  \\
G17 & 001010011011000 &  &  &  &  \\
G18 & 001010011010010 &  &  &  &  \\
G19 & 001010011011001 &  &  &  &  \\
G20 & 001010011010011 &  &  &  &  \\
G21 & 001110011011010 &  &  &  &  \\
G22 & 001110011011011 &  &  &  &  \\
G23 & 001110011001110 &  &  &  &  \\
G24 & 001110011001111 &  &  &  &  \\
G25 & 001110111001010 &  &  &  &  \\
G26 & 001110100011010 &  &  &  &  \\
G27 & 001100101011010 &  &  &  &  \\
G28 & 

In [43]:
import csv

M = A

filename = "matrix.csv"

with open(filename, "w", newline="") as f:
    writer = csv.writer(f)
    try:
        writer.writerows(M.rows())   # works if M is a Sage Matrix
    except AttributeError:
        writer.writerows(M)          # works if M is list of lists

print(f"Saved to {filename}")


Saved to matrix.csv


In [35]:
###################################
# To check for K_5 minor
###################################
K5 = graphs.CompleteGraph(5)
count=1
for G in G6:
    Gc = G.complement()

    try:
        has_minor = G.minor(K5)
    except ValueError:
        has_minor = False
    try:
        has_minor1 = Gc.minor(K5)
    except ValueError:
        has_minor1=False
    
    #checks for G and its complement, if they carry a K5 minor
    print(f"  K5-minor={has_minor}",count)
    print(f" K5-minor={has_minor1}",count)
    if has_minor or has_minor1:
        print(A[count-1])
    count+=1


  K5-minor=False 1
 K5-minor=False 1
  K5-minor=False 2
 K5-minor=False 2
  K5-minor=False 3
 K5-minor=False 3
  K5-minor=False 4
 K5-minor=False 4
  K5-minor=False 5
 K5-minor=False 5
  K5-minor=False 6
 K5-minor=False 6
  K5-minor=False 7
 K5-minor=False 7
  K5-minor=False 8
 K5-minor=False 8
  K5-minor=False 9
 K5-minor=False 9
  K5-minor=False 10
 K5-minor=False 10
  K5-minor=False 11
 K5-minor=False 11
  K5-minor=False 12
 K5-minor=False 12
  K5-minor=False 13
 K5-minor=False 13
  K5-minor=False 14
 K5-minor=False 14
  K5-minor=False 15
 K5-minor=False 15
  K5-minor=False 16
 K5-minor=False 16
  K5-minor=False 17
 K5-minor=False 17
  K5-minor=False 18
 K5-minor=False 18
  K5-minor=False 19
 K5-minor=False 19
  K5-minor=False 20
 K5-minor=False 20
  K5-minor=False 21
 K5-minor=False 21
  K5-minor=False 22
 K5-minor=False 22
  K5-minor=False 23
 K5-minor=False 23
  K5-minor=False 24
 K5-minor=False 24
  K5-minor=False 25
 K5-minor=False 25
  K5-minor=False 26
 K5-minor=False 26
  K5